<a href="https://colab.research.google.com/github/luizfreire96/Wind-energy-data-curve-fit/blob/main/Weibull_distribution_in_months.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In order to forecast wind speed in another project, it is important to analyze the data to select the best model for predictions. In this project we will analyze the data and try to find the best distribution curve for fitting the data, such as Weibull or Normal. The Data used is the same used in **Yu Ding book, Data Science for Wind Energy.**

#Importing relevant libraries

In [144]:
import pandas as pd
import numpy as np
from scipy.stats import weibull_min
import plotly.graph_objects as go

#Importing data

In [145]:
df = pd.read_csv('/content/drive/MyDrive/Wind energy collabs and datasets/Wind Time Series/Wind Time Series Dataset(10min).csv')

#Tanking a brief look at the data

In [146]:
df.describe()

,WindSpeed,Power
count,39195.000000,39195.000000
mean,6.863497,30.415774
std,2.065032,26.069511
min,0.090000,-0.007123
25%,5.370000,9.727654
50%,6.670000,22.314745
75%,8.070000,44.854827
max,19.790000,100.000000


In [147]:
df_copy = df.copy()
df_copy['Month'] = df['Time'].str[5:7]
df_copy.sample(10)

,Time,WindSpeed,Power,Month
38458,2015-10-01 00:30:00,3.48,1.105960,10
31308,2015-08-01 02:20:00,6.48,16.935171,08
26544,2015-06-22 11:30:00,4.75,4.753425,06
11874,2015-01-25 15:20:00,17.35,99.040379,01
8678,2014-12-24 17:40:00,5.32,8.221529,12
31619,2015-08-03 18:20:00,7.21,25.304981,08
11900,2015-01-25 20:00:00,12.83,99.432775,01
34937,2015-08-30 23:30:00,9.68,62.109871,08
13861,2015-02-18 08:50:00,7.82,43.631019,02
3097,2014-11-03 04:10:00,3.94,1.683545,11


In [148]:
df_copy['Month'] = pd.to_numeric(df_copy['Month'])
df_per_month = []
for i in range (1, 13, 1):
  df_per_month.append(df_copy.loc[df_copy['Month'] == i, ['Time', 'WindSpeed', 'Month']])
df_per_month2 = df_per_month.copy()

In [149]:
#grouping velocities in 0.1 m/s width bins
bins01 = [i/10 for i in range (0, 200, 1)]
labels1 = [i/10 for i in range (1, 200, 1)]
for i in df_per_month:
  i['Windspeedbin'] = pd.cut(i['WindSpeed'], bins01, labels=labels1)

In [150]:
#Counting the frequencies of each bin
df_month_count = []
for i in range (0, 12, 1):
  df_month_count.append(df_per_month[i].groupby(['Windspeedbin'], as_index=False).count())

In [151]:
meses = ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']
fig = go.Figure()
ii = 0
for i in df_month_count:
  fig.add_trace(go.Scatter(x=i['Windspeedbin'], y=i['WindSpeed']/i['WindSpeed'].sum(),
                    mode='lines',
                    name=meses[ii]))
  ii = ii+1
fig.show()


In [152]:
weibull = []
dist = []
ii = 0
x = np.linspace(0.01, 20, 200)
for i in range (0, 12, 1):
  weibull.append(weibull_min.fit(df_per_month2[i]['WindSpeed']))
  dist.append(weibull_min.pdf(x, weibull[i][0], scale=weibull[i][2], loc= weibull[i][1]) )
fig = go.Figure()
for i in range (0, 12, 1):
  fig.add_trace(go.Scatter(x=x, y=dist[i],
                    mode='lines',
                    name=meses[i]))
fig.show()